In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

In [3]:
df_hotel_traintest = pd.read_csv('tb_hotel_traintest.csv')

# EDA

## Initial

In [4]:
df_hotel_traintest.columns

Index(['hotel', 'is_cancelled', 'lead_time', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status_date', 'arrival_date', 'id_booking'],
      dtype='object')

In [5]:
df_hotel_traintest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113409 entries, 0 to 113408
Data columns (total 29 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           113409 non-null  object 
 1   is_cancelled                    113409 non-null  int64  
 2   lead_time                       113409 non-null  int64  
 3   stays_in_weekend_nights         113409 non-null  int64  
 4   stays_in_week_nights            113409 non-null  int64  
 5   adults                          113409 non-null  int64  
 6   children                        113406 non-null  float64
 7   babies                          113409 non-null  int64  
 8   meal                            113409 non-null  object 
 9   country                         112951 non-null  object 
 10  market_segment                  113409 non-null  object 
 11  distribution_channel            113409 non-null  object 
 12  is_repeated_gues

In [6]:
df_hotel_traintest.isna().mean().sort_values(ascending=False)

company                           0.943241
agent                             0.136594
country                           0.004038
children                          0.000026
hotel                             0.000000
assigned_room_type                0.000000
arrival_date                      0.000000
reservation_status_date           0.000000
total_of_special_requests         0.000000
required_car_parking_spaces       0.000000
adr                               0.000000
customer_type                     0.000000
days_in_waiting_list              0.000000
deposit_type                      0.000000
booking_changes                   0.000000
previous_bookings_not_canceled    0.000000
reserved_room_type                0.000000
is_cancelled                      0.000000
previous_cancellations            0.000000
is_repeated_guest                 0.000000
distribution_channel              0.000000
market_segment                    0.000000
meal                              0.000000
babies     

In [7]:
df_hotel_traintest.head()

,hotel,is_cancelled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date,arrival_date,id_booking
0,Resort Hotel,0,342,0,0,2,0.0,0,BB,PRT,...,NaN,NaN,0,Transient,0.0,0,0,2015-07-01,2015-07-01,0
1,Resort Hotel,0,737,0,0,2,0.0,0,BB,PRT,...,NaN,NaN,0,Transient,0.0,0,0,2015-07-01,2015-07-01,1
2,Resort Hotel,0,7,0,1,1,0.0,0,BB,GBR,...,NaN,NaN,0,Transient,75.0,0,0,2015-07-02,2015-07-01,2
3,Resort Hotel,0,13,0,1,1,0.0,0,BB,GBR,...,304.0,NaN,0,Transient,75.0,0,0,2015-07-02,2015-07-01,3
4,Resort Hotel,0,14,0,2,2,0.0,0,BB,GBR,...,240.0,NaN,0,Transient,98.0,0,1,2015-07-03,2015-07-01,4


We are going to transform the 'company' and 'agent' columns in binary columns.

In [19]:
mask_na_company = df_hotel_traintest['company'].isna()
df_hotel_traintest.loc[mask_na_company, 'is_company'] = 0
df_hotel_traintest.loc[~mask_na_company, 'is_company'] = 1
mask_na_agent = df_hotel_traintest['agent'].isna()
df_hotel_traintest.loc[mask_na_agent, 'is_agent'] = 0
df_hotel_traintest.loc[~mask_na_agent, 'is_agent'] = 1

Now, we are going to treat the remaining NA imputing values.

In [24]:
country_imputer = SimpleImputer(strategy='most_frequent')
country_imputer.fit(df_hotel_traintest[['country']])
df_hotel_traintest['country_imp'] = country_imputer.transform(df_hotel_traintest[['country']]).ravel()
children_imputer = SimpleImputer(strategy='median')
children_imputer.fit(df_hotel_traintest[['children']])
df_hotel_traintest['children_imp'] = children_imputer.transform(df_hotel_traintest[['children']]).ravel()

In [25]:
df_hotel_traintest.isna().mean().sort_values(ascending=False)

company                           0.943241
agent                             0.136594
country                           0.004038
children                          0.000026
days_in_waiting_list              0.000000
customer_type                     0.000000
adr                               0.000000
required_car_parking_spaces       0.000000
total_of_special_requests         0.000000
hotel                             0.000000
deposit_type                      0.000000
arrival_date                      0.000000
id_booking                        0.000000
is_company                        0.000000
is_agent                          0.000000
country_imp                       0.000000
reservation_status_date           0.000000
assigned_room_type                0.000000
booking_changes                   0.000000
is_cancelled                      0.000000
reserved_room_type                0.000000
previous_bookings_not_canceled    0.000000
previous_cancellations            0.000000
is_repeated

In [27]:
df_hotel_traintest.head()

,hotel,is_cancelled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,...,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date,arrival_date,id_booking,is_company,is_agent,country_imp,children_imp
0,Resort Hotel,0,342,0,0,2,0.0,0,BB,PRT,...,0.0,0,0,2015-07-01,2015-07-01,0,0.0,0.0,PRT,0.0
1,Resort Hotel,0,737,0,0,2,0.0,0,BB,PRT,...,0.0,0,0,2015-07-01,2015-07-01,1,0.0,0.0,PRT,0.0
2,Resort Hotel,0,7,0,1,1,0.0,0,BB,GBR,...,75.0,0,0,2015-07-02,2015-07-01,2,0.0,0.0,GBR,0.0
3,Resort Hotel,0,13,0,1,1,0.0,0,BB,GBR,...,75.0,0,0,2015-07-02,2015-07-01,3,0.0,1.0,GBR,0.0
4,Resort Hotel,0,14,0,2,2,0.0,0,BB,GBR,...,98.0,0,1,2015-07-03,2015-07-01,4,0.0,1.0,GBR,0.0


## Removing Columns

In [29]:
df_hotel_traintest = df_hotel_traintest.drop(['company', 'children', 'country', 'agent'], axis=1)

In [30]:
df_hotel_traintest.columns

Index(['hotel', 'is_cancelled', 'lead_time', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'babies', 'meal', 'market_segment',
       'distribution_channel', 'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type',
       'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status_date', 'arrival_date', 'id_booking', 'is_company',
       'is_agent', 'country_imp', 'children_imp'],
      dtype='object')

## Categorical Variables

In [33]:
df_hotel_traintest.select_dtypes(exclude="number").head()

,hotel,meal,market_segment,distribution_channel,reserved_room_type,assigned_room_type,deposit_type,customer_type,reservation_status_date,arrival_date,country_imp
0,Resort Hotel,BB,Direct,Direct,C,C,No Deposit,Transient,2015-07-01,2015-07-01,PRT
1,Resort Hotel,BB,Direct,Direct,C,C,No Deposit,Transient,2015-07-01,2015-07-01,PRT
2,Resort Hotel,BB,Direct,Direct,A,C,No Deposit,Transient,2015-07-02,2015-07-01,GBR
3,Resort Hotel,BB,Corporate,Corporate,A,A,No Deposit,Transient,2015-07-02,2015-07-01,GBR
4,Resort Hotel,BB,Online TA,TA/TO,A,A,No Deposit,Transient,2015-07-03,2015-07-01,GBR


In [83]:
for col in df_hotel_traintest.select_dtypes(exclude="number").columns:
    display(df_hotel_traintest[col].unique())

array(['Resort Hotel', 'City Hotel'], dtype=object)

array(['BB', 'FB', 'HB', 'SC', 'Undefined'], dtype=object)

array(['Direct', 'Corporate', 'Online TA', 'Offline TA/TO',
       'Complementary', 'Groups', 'Undefined', 'Aviation'], dtype=object)

array(['Direct', 'Corporate', 'TA/TO', 'Undefined', 'GDS'], dtype=object)

array(['C', 'A', 'D', 'E', 'G', 'F', 'H', 'L', 'P', 'B'], dtype=object)

...

array(['No Deposit', 'Refundable', 'Non Refund'], dtype=object)

array(['Transient', 'Contract', 'Transient-Party', 'Group'], dtype=object)

array(['2015-07-01', '2015-07-02', '2015-07-03', '2015-05-06',
       '2015-04-22', '2015-06-23', '2015-07-05', '2015-07-06',
       '2015-07-07', '2015-07-08', '2015-05-11', '2015-07-15',
       '2015-07-16', '2015-05-29', '2015-05-19', '2015-06-19',
       '2015-05-23', '2015-05-18', '2015-07-09', '2015-06-02',
       '2015-07-13', '2015-07-04', '2015-06-29', '2015-06-16',
       '2015-06-18', '2015-06-12', '2015-06-09', '2015-05-26',
       '2015-07-11', '2015-07-12', '2015-07-17', '2015-04-15',
       '2015-05-13', '2015-07-10', '2015-05-20', '2015-05-12',
       '2015-07-14', '2015-06-17', '2015-05-01', '2015-03-30',
       '2015-07-19', '2015-06-03', '2015-06-26', '2015-05-14',
       '2015-07-20', '2015-05-07', '2015-04-13', '2015-03-25',
       '2015-07-21', '2015-05-28', '2015-06-27', '2015-07-18',
       '2015-07-23', '2015-06-08', '2015-06-22', '2015-06-24',
       '2015-03-05', '2015-06-01', '2015-04-24', '2015-05-27',
       '2015-04-06', '2015-04-11', '2015-07-25', '2015-

array(['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04',
       '2015-07-05', '2015-07-06', '2015-07-07', '2015-07-08',
       '2015-07-09', '2015-07-10', '2015-07-11', '2015-07-12',
       '2015-07-13', '2015-07-14', '2015-07-15', '2015-07-16',
       '2015-07-17', '2015-07-18', '2015-07-19', '2015-07-20',
       '2015-07-21', '2015-07-22', '2015-07-23', '2015-07-24',
       '2015-07-25', '2015-07-26', '2015-07-27', '2015-07-28',
       '2015-07-29', '2015-07-30', '2015-07-31', '2015-08-01',
       '2015-08-02', '2015-08-03', '2015-08-04', '2015-08-05',
       '2015-08-06', '2015-08-07', '2015-08-08', '2015-08-09',
       '2015-08-10', '2015-08-11', '2015-08-12', '2015-08-13',
       '2015-08-14', '2015-08-15', '2015-08-16', '2015-08-17',
       '2015-08-18', '2015-08-19', '2015-08-20', '2015-08-21',
       '2015-08-22', '2015-08-23', '2015-08-24', '2015-08-25',
       '2015-08-26', '2015-08-27', '2015-08-28', '2015-08-29',
       '2015-08-30', '2015-08-31', '2015-09-01', '2015-

array(['PRT', 'GBR', 'USA', 'ESP', 'IRL', 'FRA', 'ROU', 'NOR', 'OMN',
       'ARG', 'POL', 'DEU', 'BEL', 'CHE', 'CN', 'GRC', 'ITA', 'NLD',
       'DNK', 'RUS', 'EST', 'CZE', 'BRA', 'FIN', 'AUS', 'SWE', 'MOZ',
       'BWA', 'LUX', 'SVN', 'ALB', 'IND', 'CHN', 'MEX', 'MAR', 'UKR',
       'SMR', 'LVA', 'PRI', 'SRB', 'CHL', 'AUT', 'BLR', 'LTU', 'TUR',
       'ZAF', 'AGO', 'ISR', 'CYM', 'CPV', 'ZWE', 'DZA', 'KOR', 'CRI',
       'HUN', 'ARE', 'TUN', 'JAM', 'HRV', 'HKG', 'IRN', 'GEO', 'AND',
       'GIB', 'URY', 'JEY', 'CAF', 'CYP', 'COL', 'GGY', 'KWT', 'NGA',
       'MDV', 'VEN', 'SVK', 'FJI', 'KAZ', 'PAK', 'IDN', 'LBN', 'PHL',
       'SEN', 'SYC', 'AZE', 'BHR', 'NZL', 'THA', 'DOM', 'MKD', 'MYS',
       'ARM', 'JPN', 'LKA', 'CUB', 'CMR', 'BIH', 'COM', 'SUR', 'UGA',
       'BGR', 'CIV', 'JOR', 'SYR', 'SGP', 'BDI', 'SAU', 'VNM', 'PLW',
       'EGY', 'PER', 'MLT', 'MWI', 'ECU', 'MDG', 'ISL', 'UZB', 'NPL',
       'BHS', 'MAC', 'TGO', 'TWN', 'DJI', 'STP', 'ETH', 'IRQ', 'MUS',
       'HND', 'RWA', 

In [77]:
# High season variable creation

In [76]:
mask_high_season = (df_hotel_traintest['arrival_date'].astype('datetime64[ns]').dt.month >= 6) & (df_hotel_traintest['arrival_date'].astype('datetime64[ns]').dt.month <= 9)
df_hotel_traintest.loc[:, 'is_high_season'] = 0
df_hotel_traintest.loc[mask_high_season, 'is_high_season'] = 1

## Pipeline

In [87]:
df_hotel_traintest.select_dtypes(include="number").head()

,is_cancelled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,id_booking,is_company,is_agent,children_imp,is_high_season
0,0,342,0,0,2,0,0,0,0,3,0,0.0,0,0,0,0.0,0.0,0.0,1.0
1,0,737,0,0,2,0,0,0,0,4,0,0.0,0,0,1,0.0,0.0,0.0,1.0
2,0,7,0,1,1,0,0,0,0,0,0,75.0,0,0,2,0.0,0.0,0.0,1.0
3,0,13,0,1,1,0,0,0,0,0,0,75.0,0,0,3,0.0,1.0,0.0,1.0
4,0,14,0,2,2,0,0,0,0,0,0,98.0,0,1,4,0.0,1.0,0.0,1.0


In [97]:
num_vars = ['lead_time', 'adr', 'adults']
cat_vars = ['hotel', 'is_high_season', 'is_company', 'country_imp', ]

# Numerical Pipeline
pt = PowerTransformer()
sca = StandardScaler()
pca = PCA()
lista_etapas = [("POWER_TRANS", pt), ("STD_SCALER", sca), ("PCA", pca)]
num_pipeline = Pipeline(lista_etapas)

# Categorical Pipeline
#ordinal = OrdinalEncoder()
ohe = OneHotEncoder(drop='first', handle_unknown='ignore', min_frequency=0.1)
cat_pipeline = Pipeline([('ONE_HOT', ohe)])

# Pipeline de Preprocessamento
pipeline_preprocessamento = [
    ("NUMERICOS", num_pipeline, num_vars),
    ("CATEGORICOS", cat_pipeline, cat_vars),
]
data_prep_pipeline = ColumnTransformer(pipeline_preprocessamento)

In [95]:
lm = LogisticRegression()
lm_pipeline = Pipeline([("PREP", data_prep_pipeline), ("LM", lm)])

## Machine Learning

In [99]:
X = df_hotel_traintest[cat_vars + num_vars]
y = df_hotel_traintest['is_cancelled']
X_train, X_test, y_train, y_test = train_test_split(X, y)

lm_pipeline.fit(X_train, y_train)

c:\Users\alexa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('PREP',
                 ColumnTransformer(transformers=[('NUMERICOS',
                                                  Pipeline(steps=[('POWER_TRANS',
                                                                   PowerTransformer()),
                                                                  ('STD_SCALER',
                                                                   StandardScaler()),
                                                                  ('PCA',
                                                                   PCA())]),
                                                  ['lead_time', 'adr',
                                                   'adults']),
                                                 ('CATEGORICOS',
                                                  Pipeline(steps=[('ONE_HOT',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 min_frequency=1))]),
                                                  ['hotel', 'is_high_season',
                                                   'is_company',
                                                   'country_imp'])])),
                ('LM', LogisticRegression())])

In [100]:
tb_log_fit = X_test.copy()

In [102]:
tb_log_fit['y_pred'] = lm_pipeline.predict(X_test)
tb_log_fit['is_cancelled'] = y_test

c:\Users\alexa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [103]:
display(tb_log_fit.sort_values(by='lead_time', ascending=False))

,hotel,is_high_season,is_company,country_imp,lead_time,adr,adults,y_pred,is_cancelled
3975,Resort Hotel,0.0,0.0,GBR,709,68.0,2,1,0
61978,City Hotel,0.0,0.0,PRT,629,62.0,2,1,1
61973,City Hotel,0.0,0.0,PRT,629,62.0,2,1,1
61971,City Hotel,0.0,0.0,PRT,629,62.0,2,1,1
57222,City Hotel,0.0,0.0,PRT,626,62.8,2,1,1
...,...,...,...,...,...,...,...,...,...
13723,Resort Hotel,0.0,1.0,PRT,0,30.0,1,0,0
19225,Resort Hotel,0.0,0.0,PRT,0,48.0,2,0,0
74576,City Hotel,0.0,0.0,KOR,0,126.0,2,0,0
20577,Resort Hotel,0.0,0.0,PRT,0,60.0,2,0,0


In [104]:
print(f'Accuracy: {np.round(accuracy_score(tb_log_fit["is_cancelled"], tb_log_fit["y_pred"]), 2) * 100}%')
print(f'Precision: {np.round(precision_score(tb_log_fit["is_cancelled"], tb_log_fit["y_pred"]), 2) * 100}%')
print(f'Recall: {np.round(recall_score(tb_log_fit["is_cancelled"], tb_log_fit["y_pred"]), 4) * 100}%')
print(f'F1-Score: {np.round(f1_score(tb_log_fit["is_cancelled"], tb_log_fit["y_pred"]), 4) * 100}%')

Accuracy: 76.0%
Precision: 73.0%
Recall: 58.589999999999996%
F1-Score: 64.84%


# Random Forest

In [110]:
rf = RandomForestClassifier(n_estimators=1000, max_depth=20)
rf_pipeline = Pipeline([("PREP", data_prep_pipeline), ("RF", rf)])

In [111]:
rf_pipeline.fit(X_train, y_train)
y_pred = rf_pipeline.predict(X_test)

c:\Users\alexa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [112]:
print(f'Accuracy: {np.round(accuracy_score(y_test, y_pred), 2) * 100}%')
print(f'Precision: {np.round(precision_score(y_test, y_pred), 2) * 100}%')
print(f'Recall: {np.round(recall_score(y_test, y_pred), 4) * 100}%')
print(f'F1-Score: {np.round(f1_score(y_test, y_pred), 4) * 100}%')

Accuracy: 81.0%
Precision: 81.0%
Recall: 62.35000000000001%
F1-Score: 70.46%


# Generating the new file for Kaggle submission

In [114]:
df_hotel_feat = pd.read_csv('tb_hotel_feat_valid_2.csv')
df_hotel_sample = pd.read_csv('tb_hotel_sample_valid.csv')

In [115]:
df_hotel_sample.head()

,id_booking,is_cancelled
0,47,0
1,132,1
2,164,0
3,171,1
4,178,0


In [116]:
df_hotel_feat.head()

,hotel,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date,arrival_date,id_booking
0,Resort Hotel,113,2,5,2,0.0,0,BB,NOR,Offline TA/TO,...,156.0,NaN,0,Transient-Party,82.88,0,2,2015-03-11,2015-07-02,47
1,Resort Hotel,5,1,0,2,0.0,0,BB,PRT,Online TA,...,240.0,NaN,0,Transient,97.00,0,0,2015-06-30,2015-07-05,132
2,Resort Hotel,81,1,3,2,2.0,0,HB,CHE,Direct,...,250.0,NaN,0,Transient,210.00,1,0,2015-04-16,2015-07-06,164
3,Resort Hotel,98,1,1,2,0.0,0,BB,USA,Online TA,...,240.0,NaN,0,Transient,87.30,1,1,2015-03-30,2015-07-06,171
4,Resort Hotel,98,1,5,2,0.0,0,HB,ESP,Online TA,...,240.0,NaN,0,Transient,117.80,1,1,2015-03-30,2015-07-06,178


In [119]:
# Transformations
mask_high_season = (df_hotel_feat['arrival_date'].astype('datetime64[ns]').dt.month >= 6) & (df_hotel_feat['arrival_date'].astype('datetime64[ns]').dt.month <= 9)
df_hotel_feat.loc[:, 'is_high_season'] = 0
df_hotel_feat.loc[mask_high_season, 'is_high_season'] = 1

mask_na_company = df_hotel_feat['company'].isna()
df_hotel_feat.loc[mask_na_company, 'is_company'] = 0
df_hotel_feat.loc[~mask_na_company, 'is_company'] = 1

mask_na_agent = df_hotel_feat['agent'].isna()
df_hotel_feat.loc[mask_na_agent, 'is_agent'] = 0
df_hotel_feat.loc[~mask_na_agent, 'is_agent'] = 1

country_imputer = SimpleImputer(strategy='most_frequent')
country_imputer.fit(df_hotel_feat[['country']])
df_hotel_feat['country_imp'] = country_imputer.transform(df_hotel_feat[['country']]).ravel()

children_imputer = SimpleImputer(strategy='median')
children_imputer.fit(df_hotel_feat[['children']])
df_hotel_feat['children_imp'] = children_imputer.transform(df_hotel_feat[['children']]).ravel()

In [121]:
y_pred = rf_pipeline.predict(df_hotel_feat[cat_vars + num_vars])

c:\Users\alexa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [130]:
df_final = df_hotel_feat[['id_booking']].copy()

In [132]:
df_final['is_cancelled'] = y_pred

In [133]:
display(df_final)

,id_booking,is_cancelled
0,47,0
1,132,0
2,164,1
3,171,0
4,178,0
...,...,...
5976,119299,0
5977,119344,0
5978,119345,0
5979,119356,0


In [134]:
display(df_hotel_sample)

,id_booking,is_cancelled
0,47,0
1,132,1
2,164,0
3,171,1
4,178,0
...,...,...
5976,119299,0
5977,119344,0
5978,119345,1
5979,119356,0


In [135]:
df_final.to_csv('df_final.csv', index=False)